<a href="https://colab.research.google.com/github/arimitramaiti/notebooks/blob/master/CNN2D_CatsDogsPandas_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

*   We initialize the Neural Network with Sequential function
*   We create 32 feature maps using a 3 by 3 feature detector. Common pratise to use 32 filters in 1st layer, then increase to 64, 128,256 in subsequent layers
*   Our input images are balck and white and we want to convert all images into 128 by 128 pixels. Hence using 128,128,3. For black&white images we can use 1 instead of 3
*   Our activation function in 1st layer is rectifier function to avoid any negative values of image pixels
*   Reducing the original size of feature map we use pooling without losing any information. In genral we take a 2 by 2 matrix
*   We add another layer of CNN with increased feature maps
*   We apply the Flattening step to get one large single vector with all information from all images
*   Add our hidden layer to convert the single large input vector from Flatten step in Fully Connected layer of ANN. A general method is 128, neither too small nor too big
*   Finally we add the output layer where we need one single answer Normal or Pneumonia with sigmoid binary function, else softmax
*   We compile using optimizer, metrics and loss as categorical_crossentropy. For exactly 2 class we can use binary_crossentropy

In [0]:
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), input_shape=(128,128,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Flatten())

cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dense(units = 3, activation = 'softmax'))

cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

*   ImageDataGenerator - Rescaling is similar to feature scaling, shear range is similar to geometrical transformation
*   flow_from_directory() code is available in https://keras.io/api/preprocessing/image/
*   target_size should have similar to input_shape dimensions mentioned in 1st CNN layer
*   class_mode is categorical for more than 2 classes of image classification

*   Mount google drive in colab
*   Separate authentication needs to be pasted from separate browser in colab

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [0]:
train_data_dir = '/gdrive/My Drive/Colab Notebooks/ImageClassification/animals/training_set'
validation_data_dir = '/gdrive/My Drive/Colab Notebooks/ImageClassification/animals/test_set'

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 color_mode="rgb",
                                                 class_mode = 'categorical',
                                                 shuffle=False)

test_set = test_datagen.flow_from_directory(validation_data_dir,
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            color_mode="rgb",
                                            class_mode = 'categorical',
                                            shuffle=False)

history = cnn.fit_generator(training_set,
                         steps_per_epoch = 800,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 200)


In [0]:
N = 25
plt.figure(figsize=(12,6))
plt.plot(np.arange(0, N), history.history['loss'], label="train_loss")
plt.plot(np.arange(0, N), history.history['val_loss'],label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title('Model Train vs Validation Loss')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.show()